# 3. Everything else, [pytest](https://docs.pytest.org/en/latest/) FTW

## pytest

> The [pytest](https://docs.pytest.org/en/latest/) framework makes it easy to write small tests, yet scales to support complex functional testing for applications and libraries.

## pytest


- Tests without boilerplate (no need for `self.assert*` or classes)

- Very powerful

- Rich plugin architecture

- Can run unittests, nosetests, doctests, works well with hypothesis

- Mature

---

*We will only take a look at a few features.*


### Example

In [1]:
%%writefile pytest/test_sample.py

def inc(x):
    return x + 1

def test_inc():
    assert inc(3) == 5

Writing pytest/test_sample.py


In [2]:
!pytest pytest/test_sample.py

============================= test session starts ==============================
platform linux -- Python 3.6.4, pytest-3.4.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/claus/repo/PYCONSK18_TESTING, inifile:
plugins: hypothesis-3.48.1
collected 1 item                                                               

pytest/test_sample.py F                                                  [100%]

=================================== FAILURES ===================================
___________________________________ test_inc ___________________________________

    def test_inc():
>       assert inc(3) == 5
E       assert 4 == 5
E        +  where 4 = inc(3)

pytest/test_sample.py:6: AssertionError
=========================== 1 failed in 0.04 seconds ===========================


### Good to Know and Do

#### [Fixtures](https://docs.pytest.org/en/latest/fixture.html#fixtures)

> The purpose of test fixtures is to provide a fixed baseline upon which tests can reliably and repeatedly execute.


In [3]:
%%writefile pytest/test_fixture.py
import pytest

@pytest.fixture
def data():
    return 'We need that'

def test_needs(data):
    assert data == 'Do we need that?'

Writing pytest/test_fixture.py


In [4]:
!pytest pytest/test_fixture.py

============================= test session starts ==============================
platform linux -- Python 3.6.4, pytest-3.4.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/claus/repo/PYCONSK18_TESTING, inifile:
plugins: hypothesis-3.48.1
collected 1 item                                                               

pytest/test_fixture.py F                                                 [100%]

=================================== FAILURES ===================================
__________________________________ test_needs __________________________________

data = 'We need that'

    def test_needs(data):
>       assert data == 'Do we need that?'
E       AssertionError: assert 'We need that' == 'Do we need that?'
E         - We need that
E         ? ^
E         + Do we need that?
E         ? ^^^^           +

pytest/test_fixture.py:8: AssertionError
=========================== 1 failed in 0.04 seconds ===========================


#### [Asserting Exceptions](https://docs.pytest.org/en/latest/assert.html#assertions-about-expected-exceptions)

In [5]:
%%writefile pytest/test_exception.py
import pytest

def fun():
    raise ZeroDivisionError()

def test_fun_raises_zero_division_error():
    with pytest.raises(TypeError):
        fun()


Writing pytest/test_exception.py


In [6]:
!pytest pytest/test_exception.py

============================= test session starts ==============================
platform linux -- Python 3.6.4, pytest-3.4.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/claus/repo/PYCONSK18_TESTING, inifile:
plugins: hypothesis-3.48.1
collected 1 item                                                               

pytest/test_exception.py F                                               [100%]

=================================== FAILURES ===================================
_____________________ test_fun_raises_zero_division_error ______________________

    def test_fun_raises_zero_division_error():
        with pytest.raises(TypeError):
>           fun()

pytest/test_exception.py:8: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

    def fun():
>       raise ZeroDivisionError()
E       ZeroDivisionError

pytest/test_exception.py:4: ZeroDivisionError
=========================== 1 failed in 0.05 seconds ===========================


#### [Parametrizing Test Functions](https://docs.pytest.org/en/latest/parametrize.html)


In [7]:
%%writefile pytest/test_parametrize.py
import pytest

def inc(x):
    return x + 1

@pytest.mark.parametrize('x, expected', [
    (1, 2),
    (4, 5),
    (10, 101),
])
def test_inc(x, expected):
    assert inc(x) == expected

Writing pytest/test_parametrize.py


In [8]:
!pytest pytest/test_parametrize.py

============================= test session starts ==============================
platform linux -- Python 3.6.4, pytest-3.4.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/claus/repo/PYCONSK18_TESTING, inifile:
plugins: hypothesis-3.48.1
collected 3 items                                                              

pytest/test_parametrize.py ..F                                           [100%]

=================================== FAILURES ===================================
_______________________________ test_inc[10-101] _______________________________

x = 10, expected = 101

    @pytest.mark.parametrize('x, expected', [
        (1, 2),
        (4, 5),
        (10, 101),
    ])
    def test_inc(x, expected):
>       assert inc(x) == expected
E       assert 11 == 101
E        +  where 11 = inc(10)

pytest/test_parametrize.py:12: AssertionError
====================== 1 failed, 2 passed in 0.05 seconds ======================


#### [Temporary Directories and Files](https://docs.pytest.org/en/latest/tmpdir.html)

In [9]:
%%writefile pytest/test_tmpdir.py

import pytest
import pandas as pd

@pytest.fixture
def dataset():
    # this is some test data we need
    df = pd.DataFrame({'a': [1, 2], 
                       'b': [2, 3]})
    return df

@pytest.fixture
def datafile(tmpdir, dataset):
    df = dataset
    path = tmpdir.join('df.csv')
    df.to_csv(path, index=None)
    return path

def test_with_tmpdir_fixture(datafile, dataset):
    pd.testing.assert_frame_equal(pd.read_csv(datafile), dataset)
    assert 0

Writing pytest/test_tmpdir.py


In [10]:
!pytest pytest/test_tmpdir.py

============================= test session starts ==============================
platform linux -- Python 3.6.4, pytest-3.4.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/claus/repo/PYCONSK18_TESTING, inifile:
plugins: hypothesis-3.48.1
collected 1 item                                                               

pytest/test_tmpdir.py F                                                  [100%]

=================================== FAILURES ===================================
___________________________ test_with_tmpdir_fixture ___________________________

datafile = local('/tmp/pytest-of-claus/pytest-8/test_with_tmpdir_fixture0/df.csv')
dataset =    a  b
0  1  2
1  2  3

    def test_with_tmpdir_fixture(datafile, dataset):
        pd.testing.assert_frame_equal(pd.read_csv(datafile), dataset)
>       assert 0
E       assert 0

pytest/test_tmpdir.py:21: AssertionError
=========================== 1 failed in 0.35 seconds ===========================


# Exercises

Implement/fix below (py)tests.

---

*You do not have to implement all requirements. Pick the ones you find most interesting!*

In [11]:
%%writefile pytest/exercises.py
"""
Complete/fix these tests.
"""
import pytest

import numpy as np
import pandas as pd


def test_comparing_list():
    assert [1, 2, 3] == [1, 3]

    
def test_comparing_dict():
    assert {'a': 1, 'b': 2} == {'c': 2, 'a': 1}

    
def test_comparing_numbers():
    assert (0.1 + 0.1 + 0.1) / 3 == 0.1

    
def test_comparing_numpy_arrays():
    assert np.array([1, 2, 3.1]) == np.array([1, 2, 3])

    
def test_comparing_dataframes():
    assert pd.DataFrame({'a': [1, 2], 'b': [3, 4]}) == pd.DataFrame({'b': [3., 4.], 'a': [1, 2]})

    
def some_exception():
    raise ZeroDivisionError('O.o')
    
    
def test_some_exception():
    # check that ZeroDivisionError is raised with 'O.o'
    pass


def multiply(a, b):
    """
    >>> multiply(1, 2)
    2
    >>> multiply([3], 3)
    [3, 3, 3]
    """
    return a * b


def test_multiply():
    # run doctest using pytest
    # make parametrized test case based on doctests
    pass


if __name__ == '__main__':
    pytest.main([__file__])

Writing pytest/exercises.py


In [12]:
!python pytest/exercises.py

============================= test session starts ==============================
platform linux -- Python 3.6.4, pytest-3.4.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/claus/repo/PYCONSK18_TESTING, inifile:
plugins: hypothesis-3.48.1
collected 7 items                                                              

pytest/exercises.py FFFFF..                                              [100%]

=================================== FAILURES ===================================
_____________________________ test_comparing_list ______________________________

    def test_comparing_list():
>       assert [1, 2, 3] == [1, 3]
E       assert [1, 2, 3] == [1, 3]
E         At index 1 diff: 2 != 3
E         Left contains more items, first extra item: 3
E         Use -v to get the full diff

pytest/exercises.py:11: AssertionError
_____________________________ test_comparing_dict ______________________________

    def test_comparing_dict():
>       assert {'a': 1, 'b': 2} == {'c': 2, 'a': 1}
E       As

# Solutions

In [15]:
# tba

# Summary

- pytest is worth learning.


- Often, most test cases are rather simple, pytest is great for that.


- Sometimes, test cases can be very complex, pytest is great for that too.

# Questions?